# Задание 1

In [8]:
import numpy as np
import pandas as pd
data2 = pd.read_csv('ml-latest-small/ratings.csv', sep=',')

In [9]:
minTime = data2.groupby('userId').min()
maxTime = data2.groupby('userId').max()
counts = data2.groupby('userId').count()

In [13]:
a = minTime.merge(maxTime, how = 'outer', on = 'userId')
b = a.merge(counts, how = 'left', on = 'userId')
logs = b.rename(columns={'rating': 'counts', 'timestamp_x': 'time_min', 'timestamp_y': 'time_max'})
d = logs[['counts', 'time_min', 'time_max']]
d

,counts,time_min,time_max
userId,,,
1,20,1260759108,1260759205
2,76,835355395,835356246
3,51,1298861589,1298932787
4,204,949778714,949982274
5,100,1163373044,1163375145
...,...,...,...
667,68,847271220,847272234
668,20,993613196,993613478
669,37,1015829081,1015829766


In [14]:
def diff(row):
    if row['counts'] >= 100:
        return (row['time_max'] - row['time_min'])
    return None

In [15]:
d['Difference'] = d.apply(diff, axis=1)
d['Difference'].mean()


<ipython-input-15-da6811fc2911>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['Difference'] = d.apply(diff, axis=1)


39331748.56653992

# Задание 2

In [1]:
import pandas as pd

In [2]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [3]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [4]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [5]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [6]:
a = auto.merge(rzd, how='outer')
n = a.merge(air, how='outer')
n.fillna(0, inplace=True)
n.sort_values(by='client_id')

,client_id,auto_revenue,rzd_revenue,air_revenue
5,111,0.0,1093.0,0.0
6,112,0.0,2810.0,0.0
0,113,57483.0,10283.0,0.0
1,114,83.0,5774.0,0.0
2,115,912.0,981.0,81.0
3,116,4834.0,0.0,4.0
4,117,98.0,0.0,13.0
7,118,0.0,0.0,173.0


In [7]:
final = n.merge(client_base, how='outer')
final.fillna(0, inplace=True)
final.sort_values(by='client_id')

,client_id,auto_revenue,rzd_revenue,air_revenue,address
5,111,0.0,1093.0,0.0,Комсомольская 4
6,112,0.0,2810.0,0.0,Энтузиастов 8а
0,113,57483.0,10283.0,0.0,Левобережная 1а
1,114,83.0,5774.0,0.0,Мира 14
2,115,912.0,981.0,81.0,ЗЖБИиДК 1
3,116,4834.0,0.0,4.0,Строителей 18
4,117,98.0,0.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4


# Задание 3


1. У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Могу предположить, что необходимо вычислить среднее значение отдельно для долготы и для широты каждого user_id. Так как земля неплоская, надо это учитывать в расчете.

2. Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга.Как это отразится на вопросе расчетах пункта 1?
чем больший вес будет у координат, находящихся рядом друг с другом, тем точнее будет расчет

3. Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.
Местоположение пользователя в момент покупки/логина в приложение